In [ ]:
import openeo # type: ignore
from rpy2 import robjects
from rpy2.robjects.packages import importr
jsonlite = importr('jsonlite')

In [ ]:
connection = openeo.connect(
    url="http://127.0.0.1:8000",
    auth_type="basic",
    auth_options={"username": "brian", "password": "123456"}
)

In [ ]:
print(connection.list_collections())

In [ ]:
print(connection.list_processes())

In [ ]:
# Load a data cube
datacube = connection.load_collection(
    collection_id="mpc/sentinel-2-l2a",
    spatial_extent={"west": -63.9, "south": -9.14, "east": -62.9, "north": -8.14},
    temporal_extent=["2022-01-01", "2022-12-31"]
)

In [ ]:
datacube = datacube.process(
    process_id="cube_regularize",
    arguments={
        "data": datacube,
        "period": "P1M",  # Monthly regularization
        "resolution": 300
    }
)

In [ ]:
datacube = datacube.ndvi(
    red="B04",
    ni="B08")

In [ ]:
# Load the RDS file using rpy2
readRDS = robjects.r['readRDS']
data_deforestation_rondonia = readRDS("./filtered_rondonia_data.rds")

In [ ]:
data_deforestation_rondonia

In [ ]:
# Serialize the data using jsonlite::serializeJSON
serializeJSON = robjects.r['serializeJSON']
# Use the function
serialized_data = serializeJSON(data_deforestation_rondonia)

In [ ]:
tempcnn_model_init = connection.mlm_class_tempcnn(
    optimizer="adam",
    learning_rate=0.0005,
    seed=42
)

In [ ]:
# Fit the model using the training dataset
tempcnn_model = tempcnn_model_init.fit(
    training_set=str(serialized_data),
    target="label"
)

In [ ]:
datacube =  tempcnn_model.predict(
    data=datacube,
    target="label",
)

In [ ]:
tempcnn_model.save_ml_model(name ="tempcnn_rondonia",
                            tasks=["classification"]
                            #options={}
                            )

In [ ]:
result = datacube.save_result(
    format="GTiff"
)

In [ ]:
job = result.create_job(
    title="Deforestation Prediction in Rondonia",
    description="Using TempCNN model to predict deforestation in Rondonia"
)

In [ ]:
job.start_and_wait()
job.get_results().download_files("output")